In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd, os, numpy as np
#from scipy.stats import skew, kurtosis, entropy
#from sklearn.mixture import GaussianMixture as gmm
#import networkx.algorithms.community as nx_comm
##import networkx as nx
#import json as js
import matplotlib.colors as mcolors
import matplotlib.ticker as mtick

In [ ]:
sns.set_style({'axes.facecolor': 'white',
         'axes.edgecolor': 'black',
        'axes.grid': False,
        'axes.spines.right': False,
        'axes.spines.top': False,
        'figure.facecolor':"white",
        'axes.labelcolor': '.15',
         'grid.color': 'white',
         'grid.linestyle': '-',
         'text.color': '.15',
         'xtick.color': '.15',
         'ytick.color': '.15',
        'patch.edgecolor': 'white',})

In [ ]:
ROOT_DIR = "/N/slate/harryan/sim_data/"
TAR_DIR = "/N/slate/harryan/sim_data_extra/"

In [ ]:
ets_files = os.listdir(TAR_DIR + "entropy_ts")
N_files = sorted([i for i in ets_files if ("N0" in i or f"N{str(1)}" in i) and "eta5" in i])


In [ ]:
colors = [
    "#abc9c8",
    "#72aeb6",
    "#4692b0",
    "#2f70a1",
    "#134b73",
    "#0a3351"
    ]
#marker_styles = ['o', 's', '^', '*', 'D', 'P']
#ets_files = os.listdir(TAR_DIR + "entropy_ts")
#N_files = sorted([i for i in ets_files if ("N0" in i or f"N{str(n)}" in i) and "eta5" in i])
def plot_entropy(n, ax, include_legend=False): 
    ets_files = os.listdir(TAR_DIR + "entropy_ts")
    N_files = sorted([i for i in ets_files if ("N0" in i or f"N{str(n)}" in i) and "eta5" in i])
    N_res = [pd.read_parquet(TAR_DIR + "entropy_ts/" + i).mean(axis = 1) for i in N_files]
    N_df = pd.concat(N_res, axis = 1)
    #print(N_df)
    N_df.columns = ["No media"] + [str(i*10)+"%" for i in [1, 3, 5, 7, 9]]
    #fig, ax = plt.subplots(1, 1)
    cs = ["coral"]+colors[1:6]
    for i, col in enumerate(N_df.columns):
        if i ==0:
            a = 1
        else:
            a = .5+(i-1)*.1
        N_df[col].plot(color=cs[i], ax=ax, label=col, alpha = a, linewidth = a) #marker=markers[i] markersize = 5
    ax.legend(title = "Audience Reach")
    ax.set_title(f"Num Media = {str(n)}")
    ax.set_ylabel("Entropy of opinions")
    ax.set_xlabel ("Time step")
    ax.set_xticks([0, 2000, 4000, 6000, 8000, 10000])
    ax.set_xticklabels(["0", "2k", "4k", "6k", "8k", "10k"])
    if include_legend == False:
        ax.get_legend().remove()
    return ax


In [ ]:
N1_ets_files = []
for i in range(1, 31):
    if i in [10, 30]:
        s = str(int(i/10))
        N1_ets_files.append(f'N1s{s}eta5.parquet')
    elif i == 20:
        s = "02"
        N1_ets_files.append(f'N1s{s}eta05.parquet')
    elif i <10:
        s = "00" + str(i)
        N1_ets_files.append(f'N1s{s}eta05.parquet')
    else:
        s = "0" + str(i)
        N1_ets_files.append(f'N1s{s}eta05.parquet')
N1_ets_files = ["N0s0eta5.parquet"] + N1_ets_files
N1_res = [pd.read_parquet(TAR_DIR + "entropy_ts/" + i).mean(axis = 1) for i in N1_ets_files]
N1_df = pd.concat(N1_res, axis = 1)
N1_df.columns = N1_df.columns = ["No media"] + [str(i)+"%" for i in range(1, 31)]
N1_res_final = [pd.read_parquet(TAR_DIR + "entropy_ts/" + i).loc["Time_9999"] for i in N1_ets_files]
N1_res_final_df = pd.concat(N1_res_final, axis = 1)
N1_res_final_df.columns =  ["No\nmedia"] + [str(i) for i in range(1, 31)]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), dpi = 300)
plot_entropy(1, ax = axes[0], include_legend= True)
axes[0].set_title("A) Effects of audience reach \n (Number of media = 1)")
ax = axes [1]
sns.pointplot(N1_res_final_df, ax = ax, errwidth = 1.5)
ax.set_ylabel("Entropy of final timestep")
ax.set_xlabel("Audience Reach (%)")
ax.set_xticks([0, 5, 10, 15, 20, 25, 30])
ax.set_xticklabels(['No media'] + [str(i) for i in range(5, 31, 5)])
ax.set_title("B) Effects of low audience reach (0 - 30%)\n (Num Media = 1)")
plt.savefig("Entropy_one_media.pdf")

In [ ]:
fig, axes = plt.subplots(1, 5, sharex= True, sharey= True, figsize=(15, 5), dpi = 300)
for i in range(5):
    ax  = axes[i]
    if i == 4:
        plot_entropy(i+1, ax = ax, include_legend=True)
    else:
        plot_entropy(i+1, ax = ax)
plt.tight_layout()
plt.suptitle("Effects of media proliferation", y = 1.07, fontsize = 16)
plt.savefig("prolif_entropy.pdf")

In [ ]:
def get_op(n, s, iteration):
    ops =[]
    folder = f"/N/slate/harryan/sim_data/N{str(n)}/s{str(s)}eta5"
    f_dir = f"{folder}/opinions/{str(iteration)}_opinions.parquet"
                #print(f_dir)
    op = pd.read_parquet(f_dir)
    return op

In [ ]:
ms = get_op(1,9, 10)["Time_10000"]
hm = get_op(5,9, 3)["Time_10000"]
pl = get_op(5,9, 84)["Time_10000"]

In [ ]:
eg = [hm, pl, ms]
eg_names = ["Homogenization", "Polarization", "Mainstreaming"]
colors = [
 "#da6c42",
 "grey",
 "#225bb2"
]
fig, axes = plt.subplots(1, 3, sharey = True, sharex = True, figsize = (9, 3), dpi = 300)
axes = axes.flatten()
for i, ax in enumerate(axes):
    sns.histplot(eg[i], ax= ax, color= colors[i], kde = True, bins = 15, alpha =.9)
    ax.set_title(eg_names[i])
    ax.set_xlabel("")
plt.suptitle("Typical final opinion distributions", y = .92)
plt.tight_layout()
plt.savefig("Typical_dist.pdf")

In [ ]:
ets_files = os.listdir(TAR_DIR + "entropy_ts")
N_files = sorted([i for i in ets_files if ("N0" in i or "s9" in i) and "eta5" in i])
N_res = [pd.read_parquet(TAR_DIR + "entropy_ts/" + i).mean(axis = 1) for i in N_files]
N_df = pd.concat(N_res, axis = 1)
N_df.columns = ["No media"] + [str(i) for i in range(1,6)]

fig, axes = plt.subplots(1, 2, figsize = (10,6), sharey = True,  dpi = 300)
ax = axes[0]
cs = ["coral"]+colors[1:6]
for i, col in enumerate(N_df.columns):
    if i ==0:
        a = 1
    else:
        a = .5+(i-1)*.1
    N_df[col].plot(color=cs[i], ax=ax, label=col) #marker=markers[i] markersize = 5
ax.legend(title = "Number of media", loc = "lower left")
ax.set_title(f"Entropy evolution")
ax.set_ylabel("Entropy of opinions")
ax.set_xlabel ("Time step")
ax.set_xticks([0, 2000, 4000, 6000, 8000, 10000])
ax.set_xticklabels(["0", "2k", "4k", "6k", "8k", "10k"])

ax = axes[1]
N90_entropies = [pd.read_parquet(TAR_DIR + "entropy_ts/" + i).loc["Time_9999"] for i in N_files]
N90_df_entropies = pd.concat(N90_entropies, axis = 1)
N90_df_entropies.columns = N_df.columns
ax, box = N90_df_entropies.boxplot(widths=0.4,
        patch_artist=True,
        notch=False,
        whis=(5, 95),
        vert=True,
        showmeans=True,
        return_type = "both", 
        positions = (1, 2, 3, 4, 5, 6),
        figsize = (8, 8),
        showfliers=False,
        ax = ax)

plt.setp(box['medians'], color='white', lw=1.2)
plt.setp(box['means'], marker='o', markerfacecolor='white', markeredgecolor='white', markersize=3)
for i, c in enumerate(cs):
    #print(i*2,i+4)
    plt.setp(box["boxes"][i], color = c, facecolor= c, alpha = .85)
    plt.setp(box['whiskers'][i*2:i*2 +2], lw=2.5, color = c, alpha = .8)
    plt.setp(box['caps'][i*2:i*2 +2], lw=2.5, color = c, alpha = .8)
ax.set_title("Entropy in the final time step")
#if include_legend == False:
    #ax.get_legend().remove() 
plt.suptitle("Effects of media proliferation\n(Audience reach = 90%)")   
plt.tight_layout()
plt.savefig("prolif_high_reach.pdf")

In [ ]:
gmm_final_list = os.listdir(TAR_DIR + "gmm_final")
gmm_final = [pd.read_parquet(TAR_DIR + f"gmm_final/{file}") for file in gmm_final_list]
gmm_final = pd.concat(gmm_final, axis = 0)
gmm_final = gmm_final.sort_values(["level_0", "level_1"])
gmm_final["N"] = gmm_final.level_0.str.replace("N", "")
gmm_final["s"] = gmm_final.level_1.apply(lambda x: x[1])
gmm_final["eta"] = gmm_final.level_1.apply(lambda x: x[-1])
gmm_final = gmm_final[gmm_final.eta != "4"]

In [ ]:
N1 = gmm_final[gmm_final.N.isin(["0", "1"])]

In [ ]:
norm = {}
for n in [0, 1]:
    for s in [0, 1, 3, 5, 7, 9]:
        cond = gmm_final[(gmm_final.N==str(n)) & (gmm_final.s==str(s))]
        if len(cond) > 0:
            #print(n,s)
            count = cond[[str(i) for i in range(100)]].T.value_counts(normalize=True)
            count.index = [t[0] for t in count.index.tolist()]
            norm[(n,s)] = {"2" : count.loc[2]}
            if 1 in count.index:
                norm[(n,s)]["1"] = count.loc[1]
            if 3 in count.index:
                norm[(n,s)][">2"] = np.around(count[count.index>2].sum(),2)
                    

In [ ]:
fig, ax = plt.subplots(1, 1, dpi = 300)
colors = [
 "#da6c42",
 "grey",
 "#225bb2"
]
pd.DataFrame(norm).T[["1", "2", ">2"]].fillna(0).plot.bar(ax = ax, color = colors, width =.8)
ax.set_ylabel("Likelihood")
ax.legend(title = "Optimal number of clusters", labels = ["Homogenization", "Polarization", "Mainstreaming"])
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1))
ax.set_xticklabels(["No media", "10%", "30%", "50%", "70%", "90%"], rotation = 0)
ax.set_xlabel("Audience Reach")
ax.set_title("Expected GMM clusters of final opinions")